In [2]:
%cd C:\Users\HP\Office_Projects\Computer_vision\Yolov5\yolov5

c:\Users\HP\Office_Projects\Computer_vision\Yolov5\yolov5


In [3]:
import cv2
import base64
import numpy as np,time
from time import time
from multiprocessing import Process
import torch, random, threading, queue
import matplotlib.pyplot as plt
import os

# fourcc = cv2.VideoWriter_fourcc(*'mpv4')
# output = cv2.VideoWriter('/Image/output_file.mp4', fourcc, 20.0, (640, 640))
path = "C:/Users/HP/Office_Projects/Computer_vision/Yolov5/yolov5/best.pt"
model = torch.hub.load('ultralytics/yolov5', 'custom', path = path, force_reload=True)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
classes = model.names

Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to C:\Users\HP/.cache\torch\hub\master.zip
YOLOv5  2022-6-6 Python-3.8.13 torch-1.11.0+cu113 CUDA:0 (NVIDIA GeForce GTX 1650, 4096MiB)

Fusing layers... 
Model summary: 213 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


In [4]:
def score_frame(frame):
    """
    Takes a single frame as input, and scores the frame using yolo5 model.
    :param frame: input frame in numpy/list/tuple format.
    :return: Labels and Coordinates of objects detected by model in the frame.
    """
    # start_time = time.time()
    model.to(device)
    frame = [frame]
    detects = model(frame)
    # inference_latency = (time.time() - start_time) * 1000
    labels, cord = detects.xyxyn[0][:, -1], detects.xyxyn[0][:, :-1]
    # print(inference_latency)
    return labels, cord

def class_to_label(x):
    """
    For a given label value, return corresponding string label.
    :param x: numeric label
    :return: corresponding string label
    """
    return classes[int(x)]

def plot_boxes(results, frame):
    """
    Takes a frame and its results as input, and plots the bounding boxes and label on to the frame.
    :param results: contains labels and coordinates predicted by model on the given frame.
    :param frame: Frame which has been scored.
    :return: Frame with bounding boxes and labels ploted on it.
    """
    labels, cord = results
    n = len(labels)
    x_shape, y_shape = frame.shape[1], frame.shape[0]
    for i in range(n):
        row = cord[i]
        if row[4] >= 0.3:
            x1, y1, x2, y2 = int(row[0] * x_shape), int(row[1] * y_shape), int(row[2] * x_shape), int(row[3] * y_shape)
            bgr = (0, 255, 0)
            cv2.rectangle(frame, (x1, y1), (x2, y2), bgr, 2)
            cv2.putText(frame, class_to_label(labels[i]), (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 0.9, bgr, 2)
    return frame

def detection(source):

    cap = cv2.VideoCapture(source)
    assert cap.isOpened()
    while True:
    
        ret, frame = cap.read()
        assert ret
        
        frame = cv2.resize(frame, (416,416))
        
        start_time = time()

        assert cap.isOpened()
        results = score_frame(frame)
        frame = plot_boxes(results, frame)
        end_time = time()
        fps = 1/np.round(end_time - start_time, 2)
        #print(f"Frames Per Second : {fps}")
            
        cv2.putText(frame, f'FPS: {int(fps)}', (20,70), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,255,0), 2)
        
        cv2.imshow('YOLOv5 Detection', frame)

        if cv2.waitKey(5) & 0xFF == 27:
            break
    
    cap.release()

In [6]:
detection('video-clip.mp4')

AssertionError: 